In [1]:
import pandas as pd
import os
import pickle
import bmra_prep
import bmra_prep.pathway_activity.prediction

In [2]:
data_dir = "/home/jing/Phd_project/project_GBM/gbm_Scripts/gbm_Scripts_bmra_u251mg/01_outputs_2020/"
#info_dir = data_dir

out_dir = "02_outputs_2020"

os.makedirs(out_dir, exist_ok = True)


# Load Data

In [3]:
# load metdadata dict and extract used elements
with open(os.path.join(data_dir, "metadata.pickle"), "rb") as f:
    all_metadata = pickle.load(f)

n_modules = all_metadata["n_modules"]
n_genes = all_metadata["n_genes"]
n_experiments = all_metadata["n_experiments"]

modules = all_metadata["modules"]
exp_ids = all_metadata["exp_ids"]
genes = all_metadata["genes"]

In [4]:
# load data
L1000_df = pd.read_csv(
    os.path.join(data_dir, "L1000_Data_norm_data.csv"),
    index_col = 0,
)

x = L1000_df.values
x.shape

(978, 77)

In [5]:
# load doses and perturbation matrix
inhib_conc_matrix = pd.read_csv(
    os.path.join(data_dir, "inhib_conc_annotated.csv"),
    index_col = 0,
).values

ic50_matrix = pd.read_csv(
    os.path.join(data_dir, "ic50_annotated.csv"),
    index_col = 0,
).values

# gamma_matrix = pd.read_csv(
#     os.path.join(data_dir, "gamma_annotated.csv"),
#     index_col = 0,
# ).values

pert_matrix = pd.read_csv(
    os.path.join(data_dir, "pert_annotated.csv"),
    index_col = 0,
).values

In [6]:
# y_true = (1 + gamma_matrix * inhib_conc_matrix / ic50_matrix) / (1 + inhib_conc_matrix / ic50_matrix)

y_true = 1 / (1 + inhib_conc_matrix / ic50_matrix)

display(y_true.shape)
y_true

(10, 77)

array([[1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        1.66666667e-01, 6.43086817e-01, 9.43396226e-01, 1.00000000e+00,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.000000

## Run models

In [7]:
a_coeffs = bmra_prep.pathway_activity.prediction.predict_coeffs(
    x, y_true, pert_matrix, 200_000, 10, 10, 10, 100)

In [8]:
a_coeffs_df = pd.DataFrame(a_coeffs, index = modules, columns = genes)
a_coeffs_df.to_csv(os.path.join(out_dir, "a_coeffs.csv"))
#a_coeffs_df = pd.read_csv(os.path.join(out_dir,'a_coeffs.csv'),index_col=0)
#a_coeffs = a_coeffs_df.values
display(a_coeffs_df.astype(bool).sum(axis='columns'))
display(a_coeffs_df)

CDK2        978
CDK4_6      978
PI3K        978
ERK         978
Estrogen    978
EGFR        978
Hypoxia     978
PDGFR       978
Aurora      978
p53         978
dtype: int64

,AARS,ABCB6,ABCC5,ABCF1,ABCF3,ABHD4,ABHD6,ABL1,ACAA1,ACAT2,...,ZMIZ1,ZMYM2,ZNF131,ZNF274,ZNF318,ZNF395,ZNF451,ZNF586,ZNF589,ZW10
CDK2,6.359700e-06,6.700031e-06,-0.000020,4.872798e-06,-0.000017,9.065138e-06,-0.000001,4.093429e-05,0.000018,-0.000017,...,0.000026,-1.779065e-05,0.000011,-0.000025,-0.000020,1.265472e-05,0.000002,3.266370e-05,0.000010,2.603910e-07
CDK4_6,-4.255990e-05,-4.076129e-07,0.000003,-8.232954e-06,-0.000006,-1.515233e-05,0.000010,8.387362e-06,0.000003,-0.000013,...,-0.000023,-6.167386e-06,0.000021,-0.000009,0.000007,2.249091e-06,-0.000011,-6.309851e-06,0.000018,2.198058e-05
PI3K,1.695356e-05,5.757387e-07,0.000011,-1.959425e-05,-0.000023,-3.017554e-07,0.000017,-4.505794e-06,-0.000013,0.000051,...,0.000022,1.122130e-05,0.000013,-0.000016,0.000020,1.081168e-05,0.000012,-1.383571e-05,0.000026,1.621861e-05
ERK,-1.941973e-05,1.260254e-05,-0.000018,-1.693284e-05,-0.000008,-7.725371e-06,0.000002,-2.304472e-06,0.000026,-0.000010,...,-0.000017,-1.113041e-05,-0.000004,-0.000014,-0.000018,7.287036e-02,-0.000008,1.959529e-06,0.000006,-9.981239e-06
Estrogen,-6.095315e-06,-1.606276e-05,-0.000020,4.259731e-06,0.000011,-8.942689e-06,0.000006,2.646067e-05,-0.000031,-0.000006,...,-0.000017,1.568597e-05,-0.000038,-0.000016,0.000012,2.361020e-03,-0.000004,-2.046752e-05,-0.000036,-2.958872e-05
EGFR,2.682972e-05,1.191333e-05,0.000014,-3.819075e-05,0.000004,-2.486480e-05,-0.000012,-1.951224e-05,0.000014,-0.000002,...,0.000001,-1.105614e-05,0.000006,0.000003,-0.000033,1.537496e-05,0.000004,2.055387e-05,0.000003,3.524878e-05
Hypoxia,-4.425278e-05,2.785224e-05,-0.000023,-3.193888e-05,-0.000023,-2.576873e-06,-0.000020,-1.547152e-05,-0.000006,-0.000016,...,-0.000017,-4.086682e-05,-0.000012,-0.000010,-0.000009,2.254338e-05,-0.000021,-4.833039e-06,0.000023,1.001497e-05
PDGFR,5.960402e-08,1.510668e-05,-0.000007,2.266954e-05,-0.000003,1.242820e-05,-0.000004,-7.116007e-07,-0.000015,0.000007,...,0.000014,1.950610e-05,-0.000006,-0.000022,0.000019,-1.799239e-05,0.000002,8.860658e-06,0.000008,-1.905074e-05
Aurora,-3.264708e-06,8.239098e-06,-0.000028,3.032336e-07,-0.000004,-2.667484e-06,0.000001,-1.031850e-05,0.000006,0.000023,...,0.000013,-2.351247e-07,0.000022,-0.000011,-0.000017,1.500043e-07,-0.000021,3.049406e-07,-0.000005,9.163997e-07
p53,2.184216e-05,4.749580e-06,-0.000031,-2.070016e-05,-0.000009,-8.465114e-06,0.000021,1.152497e-05,-0.000019,0.000021,...,0.000006,-1.983714e-05,-0.000055,-0.000002,0.000013,1.252820e-05,-0.000019,9.419328e-06,-0.000002,-4.415206e-06


In [9]:
#pathway_activity = a_coeffs @ x
#pathway_activity.shape

In [10]:
R_global = bmra_prep.pathway_activity.calc_global_response_from_pathway_activity(
    bmra_prep.pathway_activity.calc_pathway_activity(x,a_coeffs),
    modules,
    L1000_df.columns
)
R_global_df = R_global.dataframe
display(R_global_df)

,ASG002_U251MG_24H:A10,ASG002_U251MG_24H:A11,ASG002_U251MG_24H:A12,ASG002_U251MG_24H:B10,ASG002_U251MG_24H:B11,ASG002_U251MG_24H:B12,ASG002_U251MG_24H:B13,ASG002_U251MG_24H:B14,ASG002_U251MG_24H:B15,ASG002_U251MG_24H:C13,...,ASG002_U251MG_24H:M24,ASG002_U251MG_24H:N22,ASG002_U251MG_24H:N23,ASG002_U251MG_24H:N24,ASG002_U251MG_24H:O10,ASG002_U251MG_24H:O11,ASG002_U251MG_24H:O12,ASG002_U251MG_24H:P16,ASG002_U251MG_24H:P17,ASG002_U251MG_24H:P18
CDK2,0.101357,0.091644,-0.011664,0.087686,0.064857,0.091521,0.064048,0.092035,0.045154,0.030990,...,-0.006453,0.029285,0.018378,0.036078,0.073183,-0.001857,0.010707,0.053579,0.096947,-0.138181
CDK4_6,-0.034357,0.036793,0.026019,0.012811,0.022456,-0.015895,0.000758,0.015820,0.024788,0.055157,...,0.070051,-0.021989,0.054412,0.041647,0.014063,0.006247,-0.014568,-1.631479,-1.743889,-1.076619
PI3K,-1.669712,-1.764209,-1.545644,-0.174345,-0.196639,-0.207423,-0.259894,0.007741,-0.174983,-0.145027,...,0.369050,0.104077,-0.026382,0.015187,0.325997,-0.075631,-0.870298,-0.662333,-0.788470,-0.511995
ERK,-1.469459,-1.028573,-0.546271,-0.146435,-0.109047,-0.025887,-0.391860,-0.354929,-0.278114,0.161364,...,0.068806,-0.385717,-0.206668,0.015066,-0.808200,-0.089949,-0.241376,-0.971398,-1.975775,-0.884842
Estrogen,0.033600,0.083975,-0.374553,-0.134055,0.053906,-0.007647,-0.072635,0.031748,0.048555,-0.009413,...,0.019624,-0.258324,-0.042496,-0.044841,-1.658097,-1.390205,-1.384676,-0.318060,0.046600,-0.520283
EGFR,0.250455,0.395755,0.391870,-1.615584,-1.015745,0.369658,-1.616013,0.410079,-1.460245,0.244499,...,-1.670109,-1.959398,-1.464856,-1.305295,0.336689,0.353402,0.509683,-1.899952,0.306972,-1.270785
Hypoxia,-0.447556,0.141019,0.035113,0.083284,-0.046762,-0.125243,-0.202349,-0.028788,0.039342,0.068563,...,0.123890,-0.360140,0.144481,0.144646,0.104394,0.108165,-0.870917,0.027112,0.089020,-0.090083
PDGFR,-0.250544,-0.249063,-0.472509,-0.132442,-0.010820,0.044822,0.348820,0.294072,-0.282405,-0.391744,...,-0.225484,-0.052685,-0.380897,-0.390259,-0.823086,-0.056546,-0.630311,-1.859303,-1.167074,0.792055
Aurora,0.459537,-0.935912,0.284422,-1.537009,-1.989747,-1.584379,-1.529648,-1.996551,-1.651560,0.241831,...,0.024702,-0.653678,-1.512730,-1.549700,0.595733,-0.198569,0.305802,-1.064181,-0.975460,-1.058906
p53,-0.227789,0.249000,0.094384,-1.596627,-1.575652,-1.504748,-0.776855,-0.280511,0.031146,-1.635389,...,0.395466,0.140098,-0.465580,-1.084122,-0.519081,-0.979045,0.425623,0.152296,0.041817,0.005067


In [11]:
R_global_df.to_csv(os.path.join(out_dir, "R_global_annotated.csv"))
display(R_global_df)

,ASG002_U251MG_24H:A10,ASG002_U251MG_24H:A11,ASG002_U251MG_24H:A12,ASG002_U251MG_24H:B10,ASG002_U251MG_24H:B11,ASG002_U251MG_24H:B12,ASG002_U251MG_24H:B13,ASG002_U251MG_24H:B14,ASG002_U251MG_24H:B15,ASG002_U251MG_24H:C13,...,ASG002_U251MG_24H:M24,ASG002_U251MG_24H:N22,ASG002_U251MG_24H:N23,ASG002_U251MG_24H:N24,ASG002_U251MG_24H:O10,ASG002_U251MG_24H:O11,ASG002_U251MG_24H:O12,ASG002_U251MG_24H:P16,ASG002_U251MG_24H:P17,ASG002_U251MG_24H:P18
CDK2,0.101357,0.091644,-0.011664,0.087686,0.064857,0.091521,0.064048,0.092035,0.045154,0.030990,...,-0.006453,0.029285,0.018378,0.036078,0.073183,-0.001857,0.010707,0.053579,0.096947,-0.138181
CDK4_6,-0.034357,0.036793,0.026019,0.012811,0.022456,-0.015895,0.000758,0.015820,0.024788,0.055157,...,0.070051,-0.021989,0.054412,0.041647,0.014063,0.006247,-0.014568,-1.631479,-1.743889,-1.076619
PI3K,-1.669712,-1.764209,-1.545644,-0.174345,-0.196639,-0.207423,-0.259894,0.007741,-0.174983,-0.145027,...,0.369050,0.104077,-0.026382,0.015187,0.325997,-0.075631,-0.870298,-0.662333,-0.788470,-0.511995
ERK,-1.469459,-1.028573,-0.546271,-0.146435,-0.109047,-0.025887,-0.391860,-0.354929,-0.278114,0.161364,...,0.068806,-0.385717,-0.206668,0.015066,-0.808200,-0.089949,-0.241376,-0.971398,-1.975775,-0.884842
Estrogen,0.033600,0.083975,-0.374553,-0.134055,0.053906,-0.007647,-0.072635,0.031748,0.048555,-0.009413,...,0.019624,-0.258324,-0.042496,-0.044841,-1.658097,-1.390205,-1.384676,-0.318060,0.046600,-0.520283
EGFR,0.250455,0.395755,0.391870,-1.615584,-1.015745,0.369658,-1.616013,0.410079,-1.460245,0.244499,...,-1.670109,-1.959398,-1.464856,-1.305295,0.336689,0.353402,0.509683,-1.899952,0.306972,-1.270785
Hypoxia,-0.447556,0.141019,0.035113,0.083284,-0.046762,-0.125243,-0.202349,-0.028788,0.039342,0.068563,...,0.123890,-0.360140,0.144481,0.144646,0.104394,0.108165,-0.870917,0.027112,0.089020,-0.090083
PDGFR,-0.250544,-0.249063,-0.472509,-0.132442,-0.010820,0.044822,0.348820,0.294072,-0.282405,-0.391744,...,-0.225484,-0.052685,-0.380897,-0.390259,-0.823086,-0.056546,-0.630311,-1.859303,-1.167074,0.792055
Aurora,0.459537,-0.935912,0.284422,-1.537009,-1.989747,-1.584379,-1.529648,-1.996551,-1.651560,0.241831,...,0.024702,-0.653678,-1.512730,-1.549700,0.595733,-0.198569,0.305802,-1.064181,-0.975460,-1.058906
p53,-0.227789,0.249000,0.094384,-1.596627,-1.575652,-1.504748,-0.776855,-0.280511,0.031146,-1.635389,...,0.395466,0.140098,-0.465580,-1.084122,-0.519081,-0.979045,0.425623,0.152296,0.041817,0.005067
